In [2]:
from dotenv import load_dotenv
load_dotenv()
import logging
from bs4 import BeautifulSoup
import html2text
import httpx
import yaml
import json
from pydantic import Field, BaseModel
from langgraph.graph import MessagesState, StateGraph, START, END
from typing import List
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langgraph.types import Send
import operator
from typing import Annotated
from typing import NamedTuple
from utils.fetch_docs import fetch_documents
from langchain_google_genai import ChatGoogleGenerativeAI





# Set up the logger
logging.basicConfig(
    level=logging.INFO,  # Set to DEBUG for detailed logs
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        # logging.FileHandler("scraper.log"),  # Log to a file
        logging.StreamHandler()  # Log to console
    ]
)

logger = logging.getLogger(__name__)


template = """Your job is to get information from a user about what kind of agent they wish to build.

You should get the following information from them:

- What the objective of the agent is
- Various usecases of the agent 
- Some examples of what the agent will be doing (Input and expected output pairs)

If you are not able to discern this info, ask them to clarify! Do not attempt to wildly guess.

After you are able to discern all the information, call the tool AgentInstruction"""

In [4]:
class AgentInstructions(BaseModel):
    """Instructions on how to build the Agent"""
    objective: str = Field(description= "What is the primary objective of the agent")
    usecases: List[str] = Field(description= "What are the various responsibilities of the agent which it needs to fulfill")
    examples : str = Field(description= "What are some examples of the usage of the agent (input query and expected output from the agent) ?")

class AgentBuilderState(MessagesState):
    agent_instructions: AgentInstructions = Field("the requirement analysis generated by the model.")
    json_code: str = Field("The json code generated")
    python_code: str = Field("The Python code generated")

class ArchitectureEvaluationState(MessagesState):
    agent_instructions: AgentInstructions = Field("the requirement analysis generated by the model.")
    url: str = Field("url of the agent architecture to evaluate against")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-05-20", temperature=0)

def requirement_analysis_node(state: AgentBuilderState):
    
    llm_with_tool = llm.bind_tools([AgentInstructions])
    response = llm_with_tool.invoke([SystemMessage(content=template)] + state["messages"])
    
      # Construct the final answer from the arguments of the last tool call  
    if len(response.tool_calls) == 0:
        return {"messages": [response]}
    
    agent_instructions = response.tool_calls[0]
    agent_instructions = AgentInstructions(**agent_instructions["args"])
    
    return {"messages": [response], "agent_instructions": agent_instructions}


def route_state(state: AgentBuilderState):
    messages = state["messages"]
    if isinstance(messages[-1], AIMessage) and messages[-1].tool_calls:
        return "agent_kernel_builder"
    elif not isinstance(messages[-1], HumanMessage):
        return END
    return "requirement_analysis"


In [6]:

AGENT_KERNEL_PROMPT = PromptTemplate.from_template(
    """
        Task Overview:
        Design a langgraph StateGraph object implementing the the best architecture for the given set of requirements, tailored to fulfill the user requirements defined below

        <Requirements>
        Objectives: {objective}
        usecases: {responsibilities}
        examples: {examples}
        </Requirements>

        
        Expected Output:
        Your task is to produce a compiled StateGraph object.

        Guidelines for Code Generation:
        - Accuracy: Avoid hallucinations or speculative assumptions when writing code. Refer exclusively to the provided documentation.
        - Understanding: Thoroughly comprehend the architecture and examples of code.
        - Customization: Generate code tailored specifically to meet the user requirements.

    """)

def agent_kernel_builder(state: AgentBuilderState):
    """Build the agent kernel using the best architecture."""
    agent_instructions : AgentInstructions = state["agent_instructions"]
    langgraph_glossary_url = "https://langchain-ai.github.io/langgraph/concepts/low_level/"
    # agent_architecture_report.name
    #agent_architecture_report.highlights
    #agent_architecture_report.justification
    #agent_architecture_report.tailored_design
    print("reached here")
    response =  llm.invoke([HumanMessage(content=AGENT_KERNEL_PROMPT.format(
        objective=agent_instructions.objective,
        responsibilities=agent_instructions.usecases,
        examples = agent_instructions.examples,
        # langgraph_glossary=fetch_documents(langgraph_glossary_url),
        ))])
    
    # Return the generated agent kernel as the output
    return {
        "messages": [AIMessage(content="Generated agent kernel code!")],
        "python_code": response.content,
    }

In [7]:

CODE_TO_JSON_PROMPT = PromptTemplate.from_template("""
You are tasked with converting the following stategraph comPilation code into a JSON. 

Contextual documents for understanding code:
{documents}

The Input code is as follows:
{code_snippet}

OUTPUT: Explaination and JSON. Do not include any code blocks. Seperate the JSON and explaination blocks and ensure that there is an explaination for each line of JSON produced but keep the blocks seperated.
Each Output JSON will have a nodes sections containing all the nodes and an edges section

Please follow:
1. Produce the explaination first and then the JSON after it. DO not produce the JSON first. 
2. For any conditional edges, please include all the nodes that the source of a conditional edge can reach as part of the explaination.
3. Any Edge entry in the JSON can only be conditional(mention conditional: true) if the source for that edge acts as a source for multiple edges. If you cannot point to atleast 2 targets for 1 source, then that source will not have any conditional edges
4. A source can have any number of targets. Please write the explaination for each source node to target node edge
5. Please ensure that the JSON starts with __START__ node and __END__ node with the correct edges from and to them
6. Ensure all elements in the nodes sections of the output json contain the following fields: Schema_info, input_schema, output_schema, description, function_name. Please do not return any entries in the nodes without these fields and these fields can't be empty
7. Ensure all elements in the edges sections of the output json contain the following fields: source, target, routing_conditions, conditional. Please do not return any entries in the edges without these fields and they can't be empty
8. Every node should be a part of atleast one edge, Please ensure this is followed
9. Attach the code snippet for each node aswell that. Please extract it from the Input code


Example output JSON for a node:
    "code_node":{{
        "schema_info": /"/"/"CodeWriterState:
      type: TypedDict
      fields:
      - name: user_query
        type: str
      - name: execution_result
        type: str/"/"/",
    "input_schema": "CodeWriterState",
    "output_schema":"RequiremenCodeWriterStatetAnalysisState",
    "description":"This node analyzes the user_query, if the query is to write a code, it will make a tool call to run the proposed code. This node returns command object",
    "function_name": "code_step"
    }}

Example output JSON for an edge:
edge:{{ source: "abc", target: "cde", routing_condition: "if abc made a tool call then go to cde", "conditional": true}}
edge:{{ source: "abc", target: "xyz", routing_condition: "if abc made an interupt to a human then go to xyz", "conditional": true}}
edge:{{ source: "xyz", target: "_END_", routing_condition: "no nodes to go after xyz, we have our final output for this path", "conditional": false}}s         
""")
def code_to_json_node(state: AgentBuilderState):
    """Convert the generated code to JSON."""
    langgraph_glossary_url = "https://langchain-ai.github.io/langgraph/concepts/low_level/"
    json_code_ouptut = llm.invoke([HumanMessage(content=CODE_TO_JSON_PROMPT.format(
        code_snippet=state["python_code"],
        documents = fetch_documents(langgraph_glossary_url),
        ))])
    
    # Return the JSON code as the output
    return {
        "messages": [AIMessage(content="Generated JSON code!")],
        "json_code": json_code_ouptut.content,
    }



In [8]:

JSON_CODE_COMBINE_PROMPT = PromptTemplate.from_template("""
You are tasked with verifying and updating the provided JSON that represents the nodes and edges of the langgraph to ensure it is correct with respect to the input code.

Contextual Documents for Understanding Code:
{documents}

Input Code:
<Input code>{code_snippet}</Input code>
JSON:
<JSON>{node_json}</JSON>

OUTPUT: JSON
Your task is divided into two parts:
- Validation: Verify that the JSON adheres to the rules outlined below.
- Correction and Augmentation: If the JSON is incorrect or incomplete, update it with a clear justification for every change, and include the code snippet for each node extracted from the input code.

Rules for Validation and Update:
- Conditional Edges:- An edge can be marked as conditional: true only if its source acts as a source for multiple edges (i.e., at least two targets).
- If the source does not meet this condition, then it cannot have conditional edges.

- Edge Targets:- Each source can have any number of targets. This flexibility must be maintained.

- Start and End Nodes:- The JSON must begin with the __START__ node and conclude with the __END__ node, with correct edges to and from them. Edges into the END node can also be conditional if they meet the above mentioned conditions      

- Node Structure:- Each node entry in the nodes section of the JSON must include the following non-empty fields:- schema_info
- id
- schema_info
- input_schema
- output_schema
- description
- function_name


- Edge Structure:- Each edge entry in the edges section of the JSON must include the following non-empty fields:- source
- target
- routing_conditions
- conditional


- Node-Edge Relationship:- Every node must be part of at least one edge. Ensure this relationship is consistently followed.

- Node Code Snippets:- Attach a code field to every node in the JSON, extracted directly from the input code.

- Schema Requirements:- The final JSON must conform to the schema provided below:


Schema for JSON. Ensure the following schema is followed. No field should be missing for any node or edge:
- Node Example:

{{
  "code_node": {{
    "id" : "<id of the node, similar to name>"
    "schema_info": "<define the class structure of the state of this node. Also provide a name>",
    "input_schema": "<input state object name>",
    "output_schema": "<output state object name>",
    "description": "<description>",
    "function_name": "<function_name>",
    "code": "<python_code>"
  }}
}}


- Edge Example:

{{
  "edge": {{
    "source": "<source_node>",
    "target": "<target_node>",
    "routing_condition": "<routing_condition>",
    "conditional": true/false
  }}
}}


                                            
Key Instructions:
- Do not update any pre-existing field of the JSON unless you have an extremely strong justification for doing so.
- Clearly document the reasoning behind any additions, updates, or modifications to the JSON. Justifications should draw inspiration from the contextual documents mentioned earlier.
- Ensure conditional edges strictly adhere to the rules outlined above.
- Input_Schema and output_schemas can only have value None in JSON for START and END nodes. Please follow this without fail
- Include a code field for each code_node entry, with the exact code that corresponds to the node in the input code.


""")

def json_better_node(state: AgentBuilderState):
    """Add code to the json flow"""
    langgraph_glossary_url = "https://langchain-ai.github.io/langgraph/concepts/low_level/"
    json_code_ouptut = llm.invoke([HumanMessage(content=JSON_CODE_COMBINE_PROMPT.format(
        code_snippet=state["python_code"],
        documents = fetch_documents(langgraph_glossary_url),
        node_json = state["json_code"]
        ))])
    
    # Return the JSON code as the output
    return {
        "messages": [AIMessage(content="Generated updated JSON code!")],
        "json_code": json_code_ouptut.content,
    }



In [9]:
workflow = StateGraph(AgentBuilderState)
workflow.add_node("requirement_analysis", requirement_analysis_node)
workflow.add_node("agent_kernel_builder", agent_kernel_builder)
workflow.add_node("code_to_json", code_to_json_node)
workflow.add_node("json_update", json_better_node)

@workflow.add_node
def add_tool_message(state: AgentBuilderState):
    
   
    return {
        "messages": [
            ToolMessage(
                content="Requirements generated!",
                tool_call_id=state["messages"][-1].tool_calls[0]["id"],
            )
        ]
    }

workflow.add_edge("json_update", END)
workflow.add_edge("code_to_json", "json_update")
workflow.add_edge("agent_kernel_builder", "code_to_json")
workflow.add_conditional_edges("requirement_analysis", route_state, ["agent_kernel_builder", "requirement_analysis", END])
workflow.add_edge(START, "requirement_analysis")
infograph = workflow.compile()

In [10]:
import uuid

cached_human_responses = ["hi!", "rag prompt", "1 rag, 2 none, 3 no, 4 no", "red", "q"]
cached_response_index = 0
config = {"configurable": {"thread_id": str(uuid.uuid4())}}
while True:
    try:
        user = input("User (q/Q to quit): ")
    except:
        user = cached_human_responses[cached_response_index]
        cached_response_index += 1
    print(f"User (q/Q to quit): {user}")
    if user in {"q", "Q"}:
        print("AI: Byebye")
        break
    output = None
    for output in infograph.stream(
        {"messages": [HumanMessage(content=user)]}, config=config, stream_mode="updates"
    ):
        last_message = next(iter(output.values()))["messages"][-1]
        last_message.pretty_print()

    if output and "prompt" in output:
        print("Done!")
        

User (q/Q to quit): 1. Primary objective is handle customers, resolve their issues, fetch information for them and conflict to resolution 2. Use cases would be updating customer information, providing customers with the data they ask for, asking customers for their requirements and suggesting them products 3. example 1: input: what is the price of product x? output:  price of x is 300$ example 2: input: Please update my address to "delhi" output: Address updated, here are your new user details-> Name: abc, age 24, Address: Delhi Example 3: I am opening up a bakery and need to setup my supply chain and billing output: We have product x that will output your supply chain monitoring and product Y that would automate your billing and accoutning along with, I would also suggest product z which helps with customer out reach and is generally used by customers opening new business and fits your use case
================================== Ai Message ==================================
Tool Calls

2025-05-25 18:43:54,874 - INFO - HTTP Request: GET https://langchain-ai.github.io/langgraph/concepts/low_level/ "HTTP/1.1 200 OK"


================================== Ai Message ==================================

Generated JSON code!


2025-05-25 18:44:27,774 - INFO - HTTP Request: GET https://langchain-ai.github.io/langgraph/concepts/low_level/ "HTTP/1.1 200 OK"


================================== Ai Message ==================================

Generated updated JSON code!
User (q/Q to quit): q
AI: Byebye
